In [1]:
import numpy as np
from optimal_levee_bandit import load_cost_curves, prune_candidates
from beta_dueling_bandit_analytical import run_beta_dueling_bandit_analytical

# 1. Load cost curves for your city
damage_file     = "Damage_cost_curves.tab"
protection_file = "Protection_cost_curves_high_estimate.tab"
city = "Halmstad"
heights, damage_costs, protection_costs = load_cost_curves(damage_file, protection_file, city)

# 2. Define analysis horizon and prune deterministic candidates
years_range = (2025, 2100)
n_years = years_range[1] - years_range[0] + 1
candidate_indices = prune_candidates(
    heights=heights,
    protection_costs=protection_costs,
    damage_costs=damage_costs,
    n_years=n_years
)
print("Candidate heights (m):", heights[candidate_indices])

# 3. Load posterior draws and mean sea level paths
pp = np.load("pp_inputs_halmsdad_pp_mixture_2025_2100.npz")
posterior_params = {
    "eta0": pp["eta0"],
    "eta1": pp["eta1"],
    "alpha0": pp["alpha0"],
    "xi": pp["xi"],
    "u": float(pp["u"]),  # threshold in cm
}
years_future   = pp["years_future"]
X_future_paths = pp["X_future_paths"]  # shape (M_pred, T_future) in cm

# 4. Run the analytical Beta-dueling bandit
#    - delta: misidentification probability target (e.g., 0.05 for 95% confidence)
#    - max_rounds: maximum number of simulated scenarios
#    - check_every: evaluate stopping rule every this many scenarios
rng = np.random.default_rng(42)
best_height, history = run_beta_dueling_bandit_analytical(
    heights=heights,
    damage_costs=damage_costs,
    protection_costs=protection_costs,
    candidate_indices=candidate_indices,
    years_all=years_future,
    X_pred_paths_cm=X_future_paths,
    posterior_params=posterior_params,
    years_range=years_range,
    delta=0.05,
    max_rounds=50_000,
    check_every=1_000,
    rng=rng,
    verbose=True,
)

print(f"Selected design height: {best_height:.2f} m")
print("Diagnostic history:", history)


Candidate heights (m): [ 0.   0.5  1.   1.5  2.   2.5  3.   3.5  4.   4.5  5.   5.5  6.   6.5
  7.   7.5  8.   8.5  9.   9.5 10.  10.5 11.  11.5 12. ]
Round 1000: best height 2.50 m, min pairwise win prob q_best = 0.6048
Round 2000: best height 2.50 m, min pairwise win prob q_best = 0.6129
Round 3000: best height 2.50 m, min pairwise win prob q_best = 0.6249
Round 4000: best height 2.50 m, min pairwise win prob q_best = 0.6242
Round 5000: best height 2.50 m, min pairwise win prob q_best = 0.6295
Round 6000: best height 2.50 m, min pairwise win prob q_best = 0.6311
Round 7000: best height 2.50 m, min pairwise win prob q_best = 0.6292
Round 8000: best height 2.50 m, min pairwise win prob q_best = 0.6283
Round 9000: best height 2.50 m, min pairwise win prob q_best = 0.6282
Round 10000: best height 2.50 m, min pairwise win prob q_best = 0.6304
Round 11000: best height 2.50 m, min pairwise win prob q_best = 0.6322
Round 12000: best height 2.50 m, min pairwise win prob q_best = 0.6346


KeyboardInterrupt: 